##Wikipedia API

In [1]:
from wikipedia import Wikipedia
from wiki2plain import Wiki2Plain
import numpy as np

In [2]:
# print_function turns print into a python 3 function 
# this enables python to print funky text i.e. f\xn --> fun
from __future__ import (division,
                        absolute_import,
                        print_function,
                        unicode_literals
                        )

In [5]:
# will pull entire text from page 
lang = 'en' # standard english wiki pages
wiki = Wikipedia(lang)

path = "/Users/Alexander/BlueMixProject/data/"
            
subjects = ['Calculus', 
             "Trigonometry", 
             "Geometry",
             "Differential Equations", 
             "Integrals", 
             "Unit Circle", 
             "Sine", 
             "Cosine", 
             "pythagorean theorem"]

# this only needs to be ran once 
for s in subjects:
    fname = path + s + '.txt'
    try:
        raw = wiki.article(s)
    except:
        raw = None

    if raw:
        wiki2plain = Wiki2Plain(raw)
        content = wiki2plain.text
        
        with open(fname,'w') as handle:
            handle.write(content)      

In [6]:
f = open(path + subjects[-1] + '.txt','r')

In [7]:
print(f.read())

In mathematics, the Pythagorean theorem, also known as Pythagoras' theorem, is a relation in Euclidean geometry among the three sides of a right triangle. It states that the square of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the other two sides. The theorem can be written as an equation relating the lengths of the sides a, b and c, often called the "Pythagorean equation":
:a^2 + b^2 = c^2 ,
where c represents the length of the hypotenuse and a and b the lengths of the triangle's other two sides.

Although it is often argued that knowledge of the theorem predates him, the theorem is named after the ancient Greek mathematician Pythagoras ( 570 –  495 BC) as it is he who, by tradition, is credited with its first recorded proof.<!--
 --> <!--
 -->There is some evidence that Babylonian mathematicians understood the formula, although little of it indicates an application within a mathematical framework.<!--
 --><!--
 --> <!--
 -->Mesopotamian, 

##Concept Insights API

##Build Corpus

In [10]:
import json
import requests
# the ConceptInsights api from watson_developer_cloud is difficult to work with
# so I've been using request instead
from watson_developer_cloud import ConceptInsightsV2 as ConceptInsights
from pprint import pprint

In [20]:
# get account_id
base_url = "https://watson-api-explorer.mybluemix.net/concept-insights/api"
account_url = 'https://watson-api-explorer.mybluemix.net/concept-insights/api/v2/accounts'
auth=('82a698f0-8d8c-42f9-9c7d-1e4b97eebd52','50KJG6jdgZ6h')

r = requests.get(account_url, auth=auth)

In [21]:
# display account id
print(r.content)

{"accounts":[{"account_id":"liet4blerinu2"}]}


In [14]:
# create copus called 'math'
# I had a very hard time trying to get this code to work - hours lost ! 
# It still doesn't work. Maybe I should be using '.post' instead of '.put' ...?

# create_corp = "https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math"
# payload = {"id":"liet4blerinu2",
#            "access":"private",
#            "users": [{"uid":"82a698f0-8d8c-42f9-9c7d-1e4b97eebd52", "permission":"AdminReadWrite"}]}
# concept_insights_port = requests.put(create_corp,
#                                      params = payload,
#                                      auth=auth)
# concept_insights_port.status_code

In [15]:
# create corpus called 'math'
# this code only needs to run once -- otherwise 409 error will occur

!curl -D --X PUT -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h' \
    -d  '{"access":"private","users":[{"uid":"myuuid","permission":"AdminReadWrite"}]}'\
    "https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math"














{"error":"corpus already exists","url":"/api/v2/corpora/liet4blerinu2/math"}


In [17]:
# get available corpora
available_corpora = "https://gateway.watsonplatform.net" +\
                    "/concept-insights/api/v2/corpora/liet4blerinu2"

r = requests.get(available_corpora, auth=auth)
r.status_code

200

In [19]:
# display available corpora and it's parameters
print(json.loads(r.content))

{u'corpora': [{u'access': u'private', u'id': u'/corpora/liet4blerinu2/math', u'users': [{u'account_id': u'', u'permission': u'ReadWriteAdmin'}, {u'account_id': u'liet4blerinu2', u'permission': u'ReadWriteAdmin'}]}]}


In [ ]:
>>> url = 'http://httpbin.org/post'
>>> files = {'file': open('report.xls', 'rb')}

>>> r = requests.post(url, files=files)
>>> r.text

In [ ]:
# import files into 'math' corpus 
# try runing this code - provided you've made the proper substituitions 

!curl -u user:password -X PUT 'https://gateway.watsonplatform.net'+\
                              '/concept-insights/api/v2/corpora/xyz123/testCorpus/documents/testDocument' \
   -d '{
   "label": "Test document",
   "parts": [
      {
         "data": "<div><h2>Part of test document using HTML</h2></div>",
         "name": "HTML part",
         "content-type": "text/html"
      },
      {
         "data": "Part of test document pure text.",
         "name": "Text part",
         "content-type": "text/plain"
      }
   ],
   "user_fields": {
     "myField1": "my meta-data"
   }
}'

##ConceptExpansion API

In [7]:
import json
import time
from watson_developer_cloud import ConceptExpansionV1Beta as ConceptExpansion


concept_expansion = ConceptExpansion(username='82a698f0-8d8c-42f9-9c7d-1e4b97eebd52',
                                     password='50KJG6jdgZ6h')

job_id = concept_expansion.create_job(dataset='mtsamples', 
                                      seeds=['motrin', 'tylenol', 'aspirin'], 
                                      label='medications')
print(json.dumps(job_id, indent=2))

time.sleep(5)  # sleep for 5 seconds
job_status = concept_expansion.get_status(job_id)

while job_status['status'] == 'in progress' or job_status['status'] == 'awaiting work':
    time.sleep(5)  # sleep for 5 seconds
    job_status = concept_expansion.get_status(job_id)
    print(json.dumps(job_status, indent=2))

if job_status['status'] == 'done':
    results = concept_expansion.get_results(job_id)
    print(json.dumps(results, indent=2))

{
  "jobid": "M0hUQkdKOTZJWkNDSUhRSVVCT047MTI0MzAxMTE="
}
{
  "return_seeds": [
    {
      "result": "ibuprofen", 
      "prevalence": 0.8219226002693176
    }, 
    {
      "result": "advil", 
      "prevalence": 0.80472731590271
    }, 
    {
      "result": "naproxen", 
      "prevalence": 0.7841069102287292
    }, 
    {
      "result": "benadryl", 
      "prevalence": 0.7685651183128357
    }, 
    {
      "result": "acetaminophen", 
      "prevalence": 0.7564734220504761
    }, 
    {
      "result": "codeine", 
      "prevalence": 0.743308424949646
    }, 
    {
      "result": "pain relievers", 
      "prevalence": 0.7278230786323547
    }, 
    {
      "result": "laxatives", 
      "prevalence": 0.7256090641021729
    }, 
    {
      "result": "motrin ib", 
      "prevalence": 0.7229762077331543
    }, 
    {
      "result": "aleve", 
      "prevalence": 0.7223989367485046
    }, 
    {
      "result": "diuretics", 
      "prevalence": 0.7221292853355408
    }, 
    {
      "

In [ ]:
concept_expansion